# Задание 6
Выборка:

Архив записей голоса https://www.kaggle.com/rtatman/speech-accent-archive

Требуется:

Отобрать записи, соответствующие странам с минимум 30 респонеднтами в выборке

Получить сумму zero-crossing по каждой из записей

Провести ANOVA-анализ по аттрибутам родного языка, пола и возраста для уровня значимости 0.15. Дискретность признака zero-crossing игнорировать.

In [2]:
import pandas as pd       
import os 
import math 
import numpy as np
import matplotlib.pyplot as plt  
import IPython.display as ipd  # To play sound in the notebook
import librosa
import librosa.display
import os

In [3]:
import warnings
warnings.simplefilter('ignore')

In [14]:
os.chdir("../data")
print(os.listdir("../data"))

['reading-passage.txt', 'recordings', 'recordings.zip', 'speakers_all.csv']


In [15]:
df = pd.read_csv("speakers_all.csv", header=0)

print(df.shape, 'is the shape of the dataset') 
print() 
print(df.head())

(2172, 12) is the shape of the dataset

    age  age_onset              birthplace  filename native_language   sex  \
0  24.0       12.0         koussi, senegal   balanta         balanta  male   
1  18.0       10.0          buea, cameroon  cameroon        cameroon  male   
2  48.0        8.0  hong, adamawa, nigeria  fulfulde        fulfulde  male   
3  42.0       42.0   port-au-prince, haiti   haitian         haitian  male   
4  40.0       35.0   port-au-prince, haiti   haitian         haitian  male   

   speakerid   country  file_missing?  Unnamed: 9  Unnamed: 10 Unnamed: 11  
0        788   senegal           True         NaN          NaN         NaN  
1       1953  cameroon           True         NaN          NaN         NaN  
2       1037   nigeria           True         NaN          NaN         NaN  
3       1165     haiti           True         NaN          NaN         NaN  
4       1166     haiti           True         NaN          NaN         NaN  


Удалим последние 3 столбца, ибо они пустые

In [16]:
df.drop(df.columns[9:12],axis = 1, inplace = True)
print(df.columns)
df.describe()

Index(['age', 'age_onset', 'birthplace', 'filename', 'native_language', 'sex',
       'speakerid', 'country', 'file_missing?'],
      dtype='object')


,age,age_onset,speakerid
count,2172.000000,2172.000000,2172.000000
mean,33.117173,8.833333,1088.449355
std,14.453039,8.451127,628.420329
min,0.000000,0.000000,1.000000
25%,22.000000,0.000000,543.750000
50%,28.000000,8.000000,1088.500000
75%,41.000000,13.000000,1632.250000
max,97.000000,86.000000,2176.000000


In [17]:
df.head()

,age,age_onset,birthplace,filename,native_language,sex,speakerid,country,file_missing?
0,24.0,12.0,"koussi, senegal",balanta,balanta,male,788,senegal,True
1,18.0,10.0,"buea, cameroon",cameroon,cameroon,male,1953,cameroon,True
2,48.0,8.0,"hong, adamawa, nigeria",fulfulde,fulfulde,male,1037,nigeria,True
3,42.0,42.0,"port-au-prince, haiti",haitian,haitian,male,1165,haiti,True
4,40.0,35.0,"port-au-prince, haiti",haitian,haitian,male,1166,haiti,True


Выведем только те страны, чьи записи присутствуют, чтобы работать с файлами их

In [18]:
df_new = df[df['file_missing?'] == False]
df_new.country.value_counts()[:17]

usa             391
china            88
uk               67
india            58
canada           54
south korea      51
brazil           39
belgium          36
turkey           35
poland           34
australia        33
saudi arabia     33
italy            32
germany          32
ethiopia         31
russia           31
france           28
Name: country, dtype: int64

16 стран подходят, возьмем их данные

In [19]:
df_sound = df_new[(df_new.country == 'usa') | (df_new.country == 'china') | 
                  (df_new.country == 'uk') | (df_new.country == 'india') | 
                  (df_new.country == 'canada') | (df_new.country == 'south korea') | 
                  (df_new.country == 'brazil') | (df_new.country == 'belgium') | (df_new.country == 'turkey') | 
                  (df_new.country == 'poland') | (df_new.country == 'saudi arabia') | (df_new.country == 'australia') |
                  (df_new.country == 'italy') | (df_new.country == 'germany') | (df_new.country == 'ethiopia') | 
                  (df_new.country == 'russia')]
#:(
df_sound.head()

,age,age_onset,birthplace,filename,native_language,sex,speakerid,country,file_missing?
50,20.0,5.0,"addis ababa, ethiopia",amharic1,amharic,female,6,ethiopia,False
51,29.0,19.0,"gonder, ethiopia",amharic10,amharic,female,998,ethiopia,False
52,24.0,17.0,"addis ababa, ethiopia",amharic11,amharic,female,1129,ethiopia,False
53,26.0,15.0,"addis ababa, ethiopia",amharic12,amharic,female,1130,ethiopia,False
54,28.0,6.0,"addis ababa, ethiopia",amharic13,amharic,female,1131,ethiopia,False


А сейчас я добавлю в датафрейм столбец с quantity of zero crossing for each mp3 file at the whole large of one

In [28]:
from tqdm import tqdm
#for i in df_sound.filename:
#    print(i)   
list_zero_crossing = []
for i in tqdm(df_sound.filename):
    x, sr = librosa.load('recordings/' + '%s.mp3' %(i))
    n_crossings = librosa.zero_crossings(x, pad=False)
    list_zero_crossing.append(sum(n_crossings))
df_sound['zero_crossing'] = list_zero_crossing

100%|██████████████████████████████████████████████████████████████████████████████| 1045/1045 [44:15<00:00,  2.46s/it]


In [29]:
df_sound.head()

,age,age_onset,birthplace,filename,native_language,sex,speakerid,country,file_missing?,zero_crossing
50,20.0,5.0,"addis ababa, ethiopia",amharic1,amharic,female,6,ethiopia,False,67652
51,29.0,19.0,"gonder, ethiopia",amharic10,amharic,female,998,ethiopia,False,73628
52,24.0,17.0,"addis ababa, ethiopia",amharic11,amharic,female,1129,ethiopia,False,80456
53,26.0,15.0,"addis ababa, ethiopia",amharic12,amharic,female,1130,ethiopia,False,64468
54,28.0,6.0,"addis ababa, ethiopia",amharic13,amharic,female,1131,ethiopia,False,103746


Просто группируем по странам, не знаю зачем, просто, никакого смысла нет:)

In [30]:
df_sound.groupby('country').zero_crossing.sum()

country
australia        2383115
belgium          3398977
brazil           2690816
canada           3701219
china            8360087
ethiopia         2784599
germany          2288595
india            4267706
italy            2286440
poland           2702617
russia           2469554
saudi arabia     3237935
south korea      4331198
turkey           2666840
uk               4875512
usa             27701269
Name: zero_crossing, dtype: int64

In [32]:
%matplotlib inline
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

Необходимо проверить, оказывают ли выбранные факторы влияние на zero_crossing

Воспользуемся дисперсионным анализом

In [58]:
mod1 = ols('zero_crossing ~ native_language + sex + age', data=df_sound).fit()
mod1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          zero_crossing   R-squared:                       0.293
Model:                            OLS   Adj. R-squared:                  0.252
Method:                 Least Squares   F-statistic:                     7.171
Date:                Sat, 27 Apr 2019   Prob (F-statistic):           5.92e-44
Time:                        22:26:26   Log-Likelihood:                -12118.
No. Observations:                1045   AIC:                         2.435e+04
Df Residuals:                     987   BIC:                         2.464e+04
Df Model:                          57                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                      9.814e+04   6347.111     15.461      0.000    8.57e+04    1.11e+05
native_language[T.arabic]     -3343.8961   7510.279     -0.445      0.656   -1.81e+04    1.14e+04
native_language[T.armenian]   -6.348e+04   2.77e+04     -2.289      0.022   -1.18e+05   -9062.656
native_language[T.bai]         3.358e+05   2.77e+04     12.105      0.000    2.81e+05     3.9e+05
native_language[T.bavarian]   -1.911e+04   2.01e+04     -0.950      0.343   -5.86e+04    2.04e+04
native_language[T.bengali]    -4.497e+04   1.48e+04     -3.032      0.002   -7.41e+04   -1.59e+04
native_language[T.cantonese]  -1.045e+04   8276.636     -1.262      0.207   -2.67e+04    5794.493
native_language[T.dutch]      -9101.4720   7877.065     -1.155      0.248   -2.46e+04    6356.246
native_language[T.english]    -3.294e+04   6175.334     -5.335      0.000   -4.51e+04   -2.08e+04
native_language[T.farsi]       1.653e+04   2.78e+04      0.595      0.552    -3.8e+04     7.1e+04
native_language[T.finnish]    -2.102e+04   2.77e+04     -0.758      0.449   -7.54e+04    3.34e+04
native_language[T.french]     -3.024e+04   9650.086     -3.134      0.002   -4.92e+04   -1.13e+04
native_language[T.gan]         7.259e+04   2.79e+04      2.606      0.009    1.79e+04    1.27e+05
native_language[T.gedeo]      -3.456e+04   2.78e+04     -1.245      0.214    -8.9e+04    1.99e+04
native_language[T.german]     -3.509e+04   8048.709     -4.360      0.000   -5.09e+04   -1.93e+04
native_language[T.greek]       1407.0522   2.01e+04      0.070      0.944    -3.8e+04    4.08e+04
native_language[T.gujarati]   -3.147e+04   1.13e+04     -2.781      0.006   -5.37e+04   -9264.059
native_language[T.hadiyya]    -1.703e+04   2.01e+04     -0.847      0.397   -5.65e+04    2.24e+04
native_language[T.hindi]      -3.263e+04   8793.477     -3.710      0.000   -4.99e+04   -1.54e+04
native_language[T.italian]    -3.255e+04   7733.175     -4.209      0.000   -4.77e+04   -1.74e+04
native_language[T.kambaata]   -3.214e+04   1.68e+04     -1.912      0.056   -6.51e+04     846.616
native_language[T.kannada]     -4.59e+04   2.77e+04     -1.655      0.098      -1e+05    8521.250
native_language[T.khmer]       -609.0425   2.77e+04     -0.022      0.982    -5.5e+04    5.38e+04
native_language[T.kikongo]    -5.393e+04   2.77e+04     -1.944      0.052   -1.08e+05     497.820
native_language[T.konkani]    -4.674e+04   2.77e+04     -1.685      0.092   -1.01e+05    7689.757
native_language[T.korean]     -1.806e+04   7142.413     -2.528      0.012   -3.21e+04   -4043.416
native_language[T.malayalam]  -4.028e+04   1.68e+04     -2.404      0.016   -7.32e+04   -7404.601
native_language[T.mandarin]   -1.572e+04   7083.113     -2.220      0.027   -2.96e+04   -1821.612
native_language[T.marathi]    -3.338e+04   1.

In [65]:
aov_table = sm.stats.anova_lm(mod1, typ = 1)
aov_table

,df,sum_sq,mean_sq,F,PR(>F)
native_language,55.0,2.736243e+11,4.974988e+09,6.799837,5.071621e-40
sex,1.0,1.183794e+10,1.183794e+10,16.180160,6.199040e-05
age,1.0,1.359975e+10,1.359975e+10,18.588204,1.784929e-05
Residual,987.0,7.221221e+11,7.316333e+08,NaN,NaN


#### Делаем вывод, что согласно заданному уровню значимости 0.15, рассматриваемый фактор zero_crossing зависит от остальных.

# Проверим теперь межфакторное взаимодействие

In [68]:
mod2 = ols('zero_crossing ~ native_language * sex * age', data=df_sound).fit()
mod2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          zero_crossing   R-squared:                       0.361
Model:                            OLS   Adj. R-squared:                  0.271
Method:                 Least Squares   F-statistic:                     3.978
Date:                Sat, 27 Apr 2019   Prob (F-statistic):           7.18e-35
Time:                        22:36:16   Log-Likelihood:                -12064.
No. Observations:                1045   AIC:                         2.439e+04
Df Residuals:                     914   BIC:                         2.504e+04
Df Model:                         130                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
Intercept                                       3.63e+04   2.74e+04      1.326      0.185   -1.74e+04       9e+04
native_language[T.arabic]                      3259.0305   3.43e+04      0.095      0.924    -6.4e+04    7.05e+04
native_language[T.armenian]                     -83.8857     27.316     -3.071      0.002    -137.496     -30.275
native_language[T.bai]                          164.7031     16.952      9.716      0.000     131.433     197.973
native_language[T.bavarian]                   -1.441e+05      1e+06     -0.144      0.886   -2.11e+06    1.82e+06
native_language[T.bengali]                     -8.85e+04   7.21e+04     -1.227      0.220    -2.3e+05     5.3e+04
native_language[T.cantonese]                   2.104e+04   5.03e+04      0.418      0.676   -7.77e+04     1.2e+05
native_language[T.dutch]                       6.284e+04   3.79e+04      1.656      0.098   -1.16e+04    1.37e+05
native_language[T.english]                     3.394e+04   2.76e+04      1.228      0.220   -2.03e+04    8.82e+04
native_language[T.farsi]                        -12.5884     13.283     -0.948      0.344     -38.657      13.480
native_language[T.finnish]                      -10.2968     15.562     -0.662      0.508     -40.838      20.244
native_language[T.french]                      9037.5368   3.55e+04      0.255      0.799   -6.06e+04    7.87e+04
native_language[T.gan]                           23.3794      5.732      4.079      0.000      12.130      34.629
native_language[T.gedeo]                          9.9151      7.453      1.330      0.184      -4.712      24.542
native_language[T.german]                      1.326e+04    3.1e+04      0.428      0.669   -4.75e+04     7.4e+04
native_language[T.greek]                         12.6650     63.926      0.198      0.843    -112.794     138.124
native_language[T.gujarati]                    2.195e+04   5.58e+04      0.393      0.694   -8.76e+04    1.32e+05
native_language[T.hadiyya]                    -8.161e+04   4.18e+04     -1.953      0.051   -1.64e+05     412.241
native_language[T.hindi]                       2.226e+04   3.45e+04      0.646      0.519   -4.54e+04    8.99e+04
native_language[T.italian]                    -5424.1961   3.51e+04     -0.155      0.877   -7.43e+04    6.35e+04
native_language[T.kambaata]                   -8.621e+04   5.01e+04     -1.721      0.086   -1.85e+05    1.21e+04
native_language[T.kannada]                      -59.4774     41.466     -1.434      0.152    -140.856      21.901
native_language[T.khmer]                         44.8702     72.245      0.621      0.535     -96.916     186.656
native_language[T.kikongo]                     -131.6393     42.202     -3.119      0.002    -214.463     -48.815
nativ

In [69]:
sm.stats.anova_lm(mod2, typ = 1)

,df,sum_sq,mean_sq,F,PR(>F)
native_language,55.0,2.736243e+11,4.974988e+09,6.972202,1.025037e-40
sex,1.0,1.183794e+10,1.183794e+10,16.590300,5.041206e-05
native_language:sex,55.0,6.570452e+10,1.194628e+09,1.674212,1.921776e-03
age,1.0,1.195464e+10,1.195464e+10,16.753844,4.631976e-05
native_language:age,55.0,9.248487e+10,1.681543e+09,2.356600,2.524066e-07
sex:age,1.0,1.598187e+08,1.598187e+08,0.223978,6.361384e-01
native_language:sex:age,55.0,2.679466e+10,4.871756e+08,0.682753,9.623307e-01
Residual,914.0,6.521812e+11,7.135461e+08,NaN,NaN


#### Делаем вывод, что согласно заданному уровню значимости 0.15, межфакторное взаимодействие между родным языком и возрастом, а также между родным языком и полом, можно рассматривать

In [77]:
mod3 = ols('zero_crossing ~ native_language + sex + age + native_language:sex', data=df_sound).fit()
sm.stats.anova_lm(mod3, typ=1)

,df,sum_sq,mean_sq,F,PR(>F)
native_language,55.0,2.736243e+11,4.974988e+09,6.748304,1.862615e-39
sex,1.0,1.183794e+10,1.183794e+10,16.057537,6.619098e-05
native_language:sex,55.0,6.340556e+10,1.152828e+09,1.563750,6.298388e-03
age,1.0,1.210252e+10,1.210252e+10,16.416423,5.494186e-05
Residual,962.0,7.092061e+11,7.372204e+08,NaN,NaN


#### Согласно рассмотренной модели, можно сделать вывод, что есть межфакторное взаимодействием между родным языком и полом

In [82]:
mod4 = ols('zero_crossing ~ native_language + sex + age + native_language:age', data=df_sound).fit()
sm.stats.anova_lm(mod4, typ=1)

,df,sum_sq,mean_sq,F,PR(>F)
native_language,55.0,2.736243e+11,4.974988e+09,6.891090,1.927317e-40
sex,1.0,1.183794e+10,1.183794e+10,16.397295,5.552977e-05
age,1.0,1.359975e+10,1.359975e+10,18.837654,1.575320e-05
native_language:age,55.0,6.963713e+10,1.266130e+09,1.753776,7.532518e-04
Residual,953.0,6.880135e+11,7.219449e+08,NaN,NaN


#### Аналогично, есть зависимость между родным языком  и возрастом

## Рассмотрим улучшились ли модели

In [80]:
#mod3
mod3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          zero_crossing   R-squared:                       0.306
Model:                            OLS   Adj. R-squared:                  0.246
Method:                 Least Squares   F-statistic:                     5.161
Date:                Sat, 27 Apr 2019   Prob (F-statistic):           2.68e-37
Time:                        22:44:13   Log-Likelihood:                -12108.
No. Observations:                1045   AIC:                         2.438e+04
Df Residuals:                     962   BIC:                         2.479e+04
Df Model:                          82                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                  9.659e+04   8743.342     11.047      0.000    7.94e+04    1.14e+05
native_language[T.arabic]                 -4985.1834   1.12e+04     -0.443      0.658    -2.7e+04    1.71e+04
native_language[T.armenian]                -3.25e+04   1.42e+04     -2.282      0.023   -6.04e+04   -4556.325
native_language[T.bai]                     3.373e+05   2.85e+04     11.840      0.000    2.81e+05    3.93e+05
native_language[T.bavarian]               -1.761e+04   2.11e+04     -0.835      0.404    -5.9e+04    2.38e+04
native_language[T.bengali]                -4.705e+04   2.85e+04     -1.648      0.100   -1.03e+05    8969.616
native_language[T.cantonese]              -1.143e+04   1.12e+04     -1.017      0.310   -3.35e+04    1.06e+04
native_language[T.dutch]                  -1.072e+04   1.04e+04     -1.027      0.305   -3.12e+04    9761.168
native_language[T.english]                 -3.11e+04   8774.606     -3.545      0.000   -4.83e+04   -1.39e+04
native_language[T.farsi]                   1.803e+04   2.85e+04      0.632      0.527   -3.79e+04     7.4e+04
native_language[T.finnish]                -1.127e+04   1.42e+04     -0.792      0.429   -3.92e+04    1.67e+04
native_language[T.french]                 -4.146e+04   1.49e+04     -2.782      0.006   -7.07e+04   -1.22e+04
native_language[T.gan]                     3.552e+04   1.43e+04      2.485      0.013    7464.402    6.36e+04
native_language[T.gedeo]                  -1.805e+04   1.43e+04     -1.266      0.206    -4.6e+04    9921.430
native_language[T.german]                 -3.917e+04   1.09e+04     -3.578      0.000   -6.07e+04   -1.77e+04
native_language[T.greek]                  -7739.2242   2.85e+04     -0.272      0.786   -6.36e+04    4.81e+04
native_language[T.gujarati]               -3.367e+04   1.61e+04     -2.095      0.036   -6.52e+04   -2127.034
native_language[T.hadiyya]                -9278.7357   1.05e+04     -0.882      0.378   -2.99e+04    1.14e+04
native_language[T.hindi]                  -3.553e+04   1.22e+04     -2.924      0.004   -5.94e+04   -1.17e+04
native_language[T.italian]                -3.276e+04   1.25e+04     -2.622      0.009   -5.73e+04   -8236.650
native_language[T.kambaata]               -1.684e+04   8952.597     -1.881      0.060   -3.44e+04     732.019
native_language[T.kannada]                -4.437e+04   2.85e+04     -1.558      0.120      -1e+05    1.15e+04
native_language[T.khmer]                    920.3100   2.85e+04      0.032      0.974    -5.5e+04    5.68e+04
native_language[T.kikongo]                -2.772e+04   1.42e+04     -1.947      0.052   -5.57e+04     226.067
native_language[T.konkani]                -4.522e+04   2.85e+04     -1.588      0.113   -1.01e+05    1.07e+04
nat

In [81]:
#mod4
mod4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          zero_crossing   R-squared:                       0.339
Model:                            OLS   Adj. R-squared:                  0.257
Method:                 Least Squares   F-statistic:                     4.134
Date:                Sat, 27 Apr 2019   Prob (F-statistic):           3.81e-34
Time:                        22:44:21   Log-Likelihood:                -12083.
No. Observations:                1045   AIC:                         2.440e+04
Df Residuals:                     929   BIC:                         2.497e+04
Df Model:                         115                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
Intercept                                      9.963e+04   2.21e+04      4.501      0.000    5.62e+04    1.43e+05
native_language[T.arabic]                     -3.826e+04   2.57e+04     -1.490      0.137   -8.87e+04    1.21e+04
native_language[T.armenian]                    -115.1598     53.409     -2.156      0.031    -219.976     -10.343
native_language[T.bai]                          178.2099     16.745     10.643      0.000     145.348     211.072
native_language[T.bavarian]                   -2.075e+05   1.01e+06     -0.205      0.837   -2.19e+06    1.78e+06
native_language[T.bengali]                    -6.526e+04   1.43e+05     -0.458      0.647   -3.45e+05    2.15e+05
native_language[T.cantonese]                   -3.23e+04   3.68e+04     -0.877      0.381   -1.05e+05       4e+04
native_language[T.dutch]                       2.819e+04   3.01e+04      0.938      0.349   -3.08e+04    8.72e+04
native_language[T.english]                    -3.102e+04   2.23e+04     -1.394      0.164   -7.47e+04    1.26e+04
native_language[T.farsi]                          3.4835     12.734      0.274      0.784     -21.507      28.474
native_language[T.finnish]                      -17.9450     31.411     -0.571      0.568     -79.589      43.699
native_language[T.french]                     -4.381e+04   2.64e+04     -1.659      0.097   -9.56e+04    8009.428
native_language[T.gan]                           17.7136      8.738      2.027      0.043       0.565      34.862
native_language[T.gedeo]                         -8.6505     13.079     -0.661      0.509     -34.318      17.017
native_language[T.german]                     -5.491e+04   2.57e+04     -2.134      0.033   -1.05e+05   -4409.305
native_language[T.greek]                         -1.3183     77.613     -0.017      0.986    -153.635     150.998
native_language[T.gujarati]                    -2.06e+04   4.72e+04     -0.437      0.662   -1.13e+05     7.2e+04
native_language[T.hadiyya]                    -5.155e+04   7.92e+04     -0.651      0.515   -2.07e+05    1.04e+05
native_language[T.hindi]                      -4.387e+04   2.79e+04     -1.571      0.117   -9.87e+04    1.09e+04
native_language[T.italian]                    -3.411e+04   2.58e+04     -1.321      0.187   -8.48e+04    1.66e+04
native_language[T.kambaata]                   -6.076e+04   9.68e+04     -0.628      0.530   -2.51e+05    1.29e+05
native_language[T.kannada]                      -74.1477     41.682     -1.779      0.076    -155.949       7.654
native_language[T.khmer]                        -10.6318     71.472     -0.149      0.882    -150.898     129.634
native_language[T.kikongo]                     -146.0016     79.502     -1.836      0.067    -302.026      10.023
nativ

### Изменения есть, так например модель3 с добавкой native_language:sex имеет показатели выше, чем модель4 с добавкой native_language:age
    

**Вывод:** *была рассмотрена выборка из записей людей разных национальностей, в ходе которой был сделан вывод, что некоторые человеческие факторы оказывают влияние на zero crossing rate визуализации записи голосов. Более глубокий вывод сделать трудно, для этого нужно, видимо, погрузиться в "speech recognition and music information retrieval". Было интересно:)*